In [67]:
import pandas as pd
import json
import imageio
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

In [68]:
users = pd.read_csv("labeled_users.csv")
users.dropna(inplace=True)
users['race'] -= 1
raceDict = users.set_index('user_id').to_dict()['race']
users['race'].value_counts()

3.0    3114
0.0     363
1.0     234
4.0     143
2.0     134
Name: race, dtype: int64

In [69]:
with open("User demo profiles.json", encoding="utf8") as file:
    text = file.read()
    profiles = json.loads(text)

In [70]:
pics = []
labels = []
for profile in profiles:
    id = profile['id']
    if id in raceDict:
        path = profile['img_path']
        try:
            im = imageio.imread(path)
            pics.append(im)
            race = raceDict[id]
            labels.append(race)
        except FileNotFoundError:
            continue
pics = np.array(pics)
labels = np.array(labels)

In [71]:
pics.shape, labels.shape

((3396, 224, 224, 3), (3396,))

In [90]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(filters=2, kernel_size=3, activation='relu', input_shape=(224,224,3)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(5, activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 2)       56        
                                                                 
 flatten_6 (Flatten)         (None, 98568)             0         
                                                                 
 dense_45 (Dense)            (None, 100)               9856900   
                                                                 
 dense_46 (Dense)            (None, 5)                 505       
                                                                 
Total params: 9,857,461
Trainable params: 9,857,461
Non-trainable params: 0
_________________________________________________________________


In [91]:
model.fit(pics, labels, epochs=5)

Epoch 1/5
107/107 [==============================] - 15s 136ms/step - loss: 920.6637 - accuracy: 0.6266
Epoch 2/5
107/107 [==============================] - 14s 134ms/step - loss: 8.5070 - accuracy: 0.6943
Epoch 3/5
107/107 [==============================] - 14s 133ms/step - loss: 2.2498 - accuracy: 0.8271
Epoch 4/5
107/107 [==============================] - 15s 136ms/step - loss: 0.9669 - accuracy: 0.8905
Epoch 5/5
107/107 [==============================] - 14s 134ms/step - loss: 0.9193 - accuracy: 0.9317
